In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import pickle
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt, resample
import copy

from PreprocessFcns import *

%matplotlib inline

In [2]:
#save corrected version of motor scores.xls 
mot_scores = pd.read_excel('Z:CIS-PD Study\Scores\MotorTasks.xls')
#fix error in original cols labels
mot_scores.columns.values[75] = 'alternating left hand movements tremor right upper limb'
mot_scores.columns.values[150] = 'sitting tremor right upper limb'
#remove words:(Qxx) and 'rating' from each column for readability
cols= mot_scores.columns
cols = cols.tolist()
colsnew = [x.split('(')[0] for x in cols]
colsnew = [x.strip() for x in colsnew] #remove whitespace
colsnew = [x.split('rating')[0] for x in colsnew]
colsnew = [x.strip() for x in colsnew]
colsnew = [x.lower() for x in colsnew] #make all lower case 
colsnew = [x.replace('\x97',' ') for x in colsnew]
colsnew = [x.replace('—',' ') for x in colsnew]    
#simplify notation
for i in range(len(colsnew)):
    x = colsnew[i]
    if x.find('finger to nose')>-1:
        colsnew[i] = x.replace(' hand','')
    if x.find('alternating')>-1:
        colsnew[i] = x.replace(' hand movements','')
c = dict(zip(cols,colsnew))
mot_scores = mot_scores.rename(index=str, columns=c)
#change 1 month label to 4 wks for proper sorting (to incorporate in xls file)
mot_scores.loc[mot_scores['visit']=='1 Month','visit']='4 Weeks'
mot_scores.to_excel('Z:CIS-PD Study\Scores\MotorTasks.xls') #note that timestamps are not properly converted

In [33]:
def DataAggregator(subjDict,taskList_Abb,taskScores,sessionList):
    
    Data = pd.DataFrame()
    
    s = 0
    for subj in list(subjDict.keys()):
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subjDict)))
        
        for t in range(len(taskList_Abb)):
            
            task = taskList_Abb[t]
            task_score = taskScores[t]
            
            for trial in range(len(sessionList)):
                
                visit = sessionList[trial]
                
                try:
                    data = pd.read_csv('Z:CIS-PD Study\\MJFF Curation\\TaskAcc\\' + str(subj) + '_' + str(trial) + 
                                      '_' + task + '.csv',parse_dates=['timestamp'])[['timestamp','x','y','z']]
                
                except:
                    #print('No data found for %s trial %d'%(task,trial))
                    continue
                    
                side = subjDict[subj]
                
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                            task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                            task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
                subj_score.index = range(len(subj_score))
                    
                data['timestamp'] = (data.timestamp.values - data.timestamp.values[0]).astype('timedelta64[ms]').astype(int)
                data = data.set_index('timestamp')
                
                data = HPfilter(data)
                
                clip_data = gen_clips(data,downsample=35,basefreq=50)
                feature_extraction(clip_data)
                
                if 'features' in clip_data.keys():
                    D = clip_data['features']
                    featcols = list(D.columns)
                    D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                    D['Tremor'] = subj_score['Tremor'][trial]
                    D['Visit'] = visit
                    D['Task'] = task
                    D['Subject'] = subj
                    Data = pd.concat([Data,D])
                    
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    Data.to_csv('Z:CIS-PD Study\\Smartwatch Data 35Hz.csv')

In [34]:
subjDict = {1004:'right',1016:'left',1018:'left',1019:'left',1020:'right',1024:'left',1029:'left',1030:'left',1032:'left',
            1038:'left',1044:'right',1046:'right',1049:'left',1051:'left'}
#subjDict = {1004:'right'}
#1047

taskList = ['Standing', 'Walking', 'Walking while counting', 'Finger to nose--right hand', 
            'Finger to nose--left hand', 'Alternating right hand movements', 'Alternating left hand movements', 
            'Sit to stand', 'Drawing on a paper', 'Typing on a computer keyboard', 'Assembling nuts and bolts', 
            'Taking a glass of water and drinking', 'Organizing sheets in a folder', 'Folding towels', 'Sitting']

taskScores = ['standing','walking','walking while counting','finger to nose right','finger to nose left',
                   'alternating right','alternating left','sit to stand','drawing on a paper',
                   'typing on a computer keyboard','assembling nuts and bolts','taking a glass of water and drinking',
                   'organizing sheets in a folder','folding towels','sitting']

taskList_Abb = ['Stndg', 'Wlkg', 'WlkgCnt', 'FtnR', 'FtnL', 'RamR', 'RamL', 'SitStand', 'Drwg', 'Typg', 'NtsBts',
                'Drnkg', 'Sheets', 'Fldg', 'Sitng']

sessionList = ['2 Weeks: Time 0', '2 Weeks: Time 30', '2 Weeks: Time 60', '2 Weeks: Time 90', '2 Weeks: Time 120', 
               '2 Weeks: Time 150', '4 Weeks']

In [35]:
DataAggregator(subjDict,taskList_Abb,taskScores,sessionList)

Subject 1004 (1 of 14)


C:\Users\Public\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\Public\Documents\Anaconda\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)
